In [3]:
import os

os.chdir(
    r"C:\Users\EM\OneDrive\Desktop\DA course\Python\Hackathon\Flatten_Covid19_Dataset-20251217T221337Z-3-001\Flatten_Covid19_Dataset"
)



In [4]:
print(os.getcwd())
print(os.listdir())

C:\Users\EM\OneDrive\Desktop\DA course\Python\Hackathon\Flatten_Covid19_Dataset-20251217T221337Z-3-001\Flatten_Covid19_Dataset
['flatten_covid_cleaned_master.csv', 'ontario_covid_combined_cleaned.csv', 'schema_1_ontario_final.csv', 'schema_2_ontario_final.csv', 'schema_3_ontario_final.csv']


In [5]:
import pandas as pd

# Reading the file schema_1_ontario_final
df_schema1 = pd.read_csv("schema_1_ontario_final.csv")

In [6]:
#Reading the files
df_schema2=pd.read_csv("schema_2_ontario_final.csv")
#Reading the file schema_3_ontario_final
df_schema3=pd.read_csv("schema_3_ontario_final.csv")

In [7]:
for df in [df_schema1, df_schema2, df_schema3]:
    df.columns = df.columns.str.strip().str.lower()


In [8]:
print(df_schema1.columns.tolist())
print(df_schema2.columns.tolist())
print(df_schema3.columns.tolist())


['week', 'fsa', 'probable', 'vulnerable', 'is_most_recent', 'fever_chills_shakes', 'cough', 'shortness_of_breath', 'over_60', 'any_medical_conditions', 'travel_outside_canada', 'contact_with_illness']
['week', 'fsa', 'probable', 'vulnerable', 'is_most_recent', 'fever_chills_shakes', 'cough', 'shortness_of_breath', 'any_medical_conditions', 'travel_outside_canada', 'contact_with_illness', 'symptoms', 'conditions', 'ethnicity', 'sex', 'needs', 'age_1']
['month', 'fsa', 'probable', 'vulnerable', 'fever_chills_shakes', 'cough', 'shortness_of_breath', 'any_medical_conditions', 'travel_outside_canada', 'contact_with_illness', 'contact_in_household', 'tested', 'covid_results_date', 'covid_positive', 'symptoms', 'conditions', 'ethnicity', 'sex', 'needs', 'age_1', 'mental_health_impact', 'travel_work_school', 'self_isolating', 'media_channels', 'financial_obligations_impact', 'tobacco_usage']


In [9]:
master_columns = sorted(list(set(df_schema1.columns) | set(df_schema2.columns) | set(df_schema3.columns)))
print(master_columns)


['age_1', 'any_medical_conditions', 'conditions', 'contact_in_household', 'contact_with_illness', 'cough', 'covid_positive', 'covid_results_date', 'ethnicity', 'fever_chills_shakes', 'financial_obligations_impact', 'fsa', 'is_most_recent', 'media_channels', 'mental_health_impact', 'month', 'needs', 'over_60', 'probable', 'self_isolating', 'sex', 'shortness_of_breath', 'symptoms', 'tested', 'tobacco_usage', 'travel_outside_canada', 'travel_work_school', 'vulnerable', 'week']


In [10]:
#Add missing columns to each DataFrame
import numpy as np

def align_schema(df, master_cols):
    for col in master_cols:
        if col not in df.columns:
            df[col] = np.nan
    return df[master_cols]

df_schema1 = align_schema(df_schema1, master_columns)
df_schema2 = align_schema(df_schema2, master_columns)
df_schema3 = align_schema(df_schema3, master_columns)


In [11]:
#Combine all files into one master table
flatten_master = pd.concat([df_schema1, df_schema2, df_schema3], ignore_index=True)
flatten_master.shape
flatten_master.head()


,age_1,any_medical_conditions,conditions,contact_in_household,contact_with_illness,cough,covid_positive,covid_results_date,ethnicity,fever_chills_shakes,...,self_isolating,sex,shortness_of_breath,symptoms,tested,tobacco_usage,travel_outside_canada,travel_work_school,vulnerable,week
0,NaN,n,NaN,NaN,n,n,NaN,NaN,NaN,n,...,NaN,NaN,n,NaN,NaN,NaN,n,NaN,n,13.0
1,NaN,y,NaN,NaN,y,n,NaN,NaN,NaN,n,...,NaN,NaN,n,NaN,NaN,NaN,n,NaN,y,13.0
2,NaN,n,NaN,NaN,n,n,NaN,NaN,NaN,n,...,NaN,NaN,n,NaN,NaN,NaN,n,NaN,y,14.0
3,NaN,n,NaN,NaN,n,n,NaN,NaN,NaN,n,...,NaN,NaN,n,NaN,NaN,NaN,n,NaN,n,13.0
4,NaN,y,NaN,NaN,n,n,NaN,NaN,NaN,n,...,NaN,NaN,n,NaN,NaN,NaN,n,NaN,y,13.0


In [12]:
#Clean survey responses
yes_no_cols = [
    "probable", "vulnerable", "fever_chills_shakes", "cough",
    "shortness_of_breath", "any_medical_conditions",
    "travel_outside_canada", "contact_with_illness",
    "contact_in_household", "tested", "covid_positive",
    "self_isolating"
]

for col in yes_no_cols:
    if col in flatten_master.columns:
        flatten_master[col] = flatten_master[col].str.lower().map({"yes": 1, "no": 0})


In [13]:
#Handle demographics
flatten_master["age_1"] = flatten_master["age_1"].astype("category")
flatten_master["sex"] = flatten_master["sex"].astype("category")
flatten_master["ethnicity"] = flatten_master["ethnicity"].astype("category")


In [14]:
#Check missing values
flatten_master.isna().mean().sort_values(ascending=False)


any_medical_conditions          1.000000
contact_with_illness            1.000000
contact_in_household            1.000000
covid_positive                  1.000000
cough                           1.000000
fever_chills_shakes             1.000000
tested                          1.000000
vulnerable                      1.000000
travel_outside_canada           1.000000
shortness_of_breath             1.000000
self_isolating                  1.000000
probable                        1.000000
covid_results_date              0.998569
financial_obligations_impact    0.996885
symptoms                        0.991197
media_channels                  0.980612
conditions                      0.979065
ethnicity                       0.974842
tobacco_usage                   0.970494
travel_work_school              0.970487
mental_health_impact            0.970463
month                           0.947182
needs                           0.923038
sex                             0.919709
age_1           

In [15]:
flatten_master.to_csv("flatten_covid_cleaned_master.csv", index=False)


In [16]:
import os
print(os.getcwd())

C:\Users\EM\OneDrive\Desktop\DA course\Python\Hackathon\Flatten_Covid19_Dataset-20251217T221337Z-3-001\Flatten_Covid19_Dataset


In [18]:
flatten_master.to_csv(
    "ontario_covid_combined_cleaned.csv",
    index=False
)